<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Import" data-toc-modified-id="Import-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Import</a></span></li></ul></li><li><span><a href="#EDA" data-toc-modified-id="EDA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>EDA</a></span></li></ul></div>

# Customers Segmentation

### Introduction

### Import

In [1]:
import numpy as np
from numpy import where
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)

import warnings
from collections import Counter
warnings.filterwarnings('ignore')

from sqlalchemy import create_engine

import sys
from pathlib import Path
import os
os.getcwd()

'/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Customer_segmentation'

In [2]:
# Visualizations libraries
import matplotlib.ticker as mtick
import matplotlib.gridspec as grid_spec
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
# Data pre-processing 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


# Modelling 
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn import svm
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import VotingClassifier
import xgboost as xgb


# Evaluation & CV
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold
from sklearn.model_selection import KFold, cross_val_score


# pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


# SMOTE
from imblearn.over_sampling import BorderlineSMOTE




## EDA

In [4]:
path = Path('/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Customer_segmentation/Data')

In [61]:
df_train = pd.read_csv(path/'Train.csv')
df_test = pd.read_csv(path/'Test.csv')

In [62]:
df_train.shape, df_test.shape

((8068, 11), (2627, 10))

In [63]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [64]:
df_train.isnull().sum()/len(df_train)

ID                 0.000000
Gender             0.000000
Ever_Married       0.017353
Age                0.000000
Graduated          0.009668
Profession         0.015369
Work_Experience    0.102752
Spending_Score     0.000000
Family_Size        0.041522
Var_1              0.009420
Segmentation       0.000000
dtype: float64

In [65]:
df_train.Segmentation.value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

In [66]:
df_train['Var_1'].value_counts()

Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: Var_1, dtype: int64

In [67]:
df_train.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [68]:
df_train.Profession.value_counts()

Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: Profession, dtype: int64

In [19]:
df_train.columns

Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')

In [70]:
cats = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1', 'Segmentation']
nums = ['Age', 'Work_Experience', 'Family_Size']

for col in cats:
    df_train[col] = df_train[col].fillna('Unknown')
    
for col in nums:
    df_train[col] = df_train[col].fillna(999)

In [76]:
# There are 16 rows which Graduated=='No' and Profession=='Unknown' & Age<24
# Even though there is no Students in Profession, I would like to create the value for students
df_train['Profession'].loc[(df_train['Graduated']=='No') & (df_train['Profession']=='Unknown') & (df_train['Age']<24)] = 'Students'

In [82]:
df_train['Profession'].value_counts()

Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Unknown           108
Students           16
Name: Profession, dtype: int64

In [83]:
df_train.Spending_Score.value_counts()

Low        4878
Average    1974
High       1216
Name: Spending_Score, dtype: int64

In [85]:
df_train.Age.max()

89

In [86]:
# Create Age group - Young, Adult, Senior
bins = [18, 22, 50, 90]
labels = ['Young', 'Adult', 'Senior']
df_train['Age_range'] = pd.cut(df_train['Age'], bins=bins, labels=labels, right=False)

In [88]:
# Age banding
def age_band(num):
    for i in range(1, 100):
        if num < 10*i:
            return f'{(i-1)*10} ~ {i*10}'
df_train['Age_band'] = df_train['Age'].apply(age_band)

In [89]:
df_train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,Age_range,Age_band
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D,Adult,20 ~ 30
1,462643,Female,Yes,38,Yes,Engineer,999.0,Average,3.0,Cat_4,A,Adult,30 ~ 40
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B,Senior,60 ~ 70
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B,Senior,60 ~ 70
4,462669,Female,Yes,40,Yes,Entertainment,999.0,High,6.0,Cat_6,A,Adult,40 ~ 50


In [91]:
df_train['Work_Experience'].value_counts()

1.0      2354
0.0      2318
999.0     829
9.0       474
8.0       463
2.0       286
3.0       255
4.0       253
6.0       204
7.0       196
5.0       194
10.0       53
11.0       50
12.0       48
13.0       46
14.0       45
Name: Work_Experience, dtype: int64